In [61]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [62]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [63]:
import tensorflow as tf
from tensorflow import keras

print("TensorFlow Version:", tf.__version__)
print("Keras Version:", keras.__version__)

TensorFlow Version: 2.18.0
Keras Version: 3.8.0


In [64]:
import os
import shutil
import random
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define dataset path
dataset_path = "C:\\Users\\megha\\OneDrive\\Desktop\\Major Project\\Dataset\\multi_dataset"  # Update with your dataset directory path
TARGET_SIZE = 600  # Minimum number of images per class

# Function to check class distribution
def check_class_distribution(dataset_path):
    class_counts = {}
    for class_name in os.listdir(dataset_path):
        class_dir = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_dir):  # Ensure it's a folder
            num_images = len(os.listdir(class_dir))
            class_counts[class_name] = num_images
    return class_counts

# Function to apply oversampling (copying images)
def oversample_class(class_dir, target_size):
    current_images = os.listdir(class_dir)
    current_count = len(current_images)

    while current_count < target_size:
        file = random.choice(current_images)  # Pick a random image
        new_file = f"copy_{current_count}_{file}"
        shutil.copy(os.path.join(class_dir, file), os.path.join(class_dir, new_file))
        current_count += 1
    print(f"✅ Oversampled {os.path.basename(class_dir)} to {target_size} images")

# Function to apply data augmentation
def augment_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))  # Resize to match model input size

    # Apply random augmentation
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )

    img = np.expand_dims(img, axis=0)  # Expand dimension for augmentation
    it = datagen.flow(img, batch_size=1)

    augmented_img = next(it)[0].astype(np.uint8)  # Get augmented image
    return augmented_img

# Function to balance dataset using oversampling and augmentation
def balance_dataset(dataset_path, target_size=TARGET_SIZE):
    class_counts = check_class_distribution(dataset_path)

    for class_name, num_images in class_counts.items():
        class_dir = os.path.join(dataset_path, class_name)

        if num_images < target_size:
            print(f"⚠️ Class {class_name} has only {num_images} images. Balancing...")
            
            while num_images < target_size:
                file = random.choice(os.listdir(class_dir))
                img_path = os.path.join(class_dir, file)

                if num_images % 2 == 0:
                    # Oversampling (Copy existing images)
                    new_file = f"copy_{num_images}_{file}"
                    shutil.copy(img_path, os.path.join(class_dir, new_file))
                else:
                    # Data Augmentation (Generate new variations)
                    augmented_img = augment_image(img_path)
                    new_file = f"aug_{num_images}.jpg"
                    cv2.imwrite(os.path.join(class_dir, new_file), augmented_img)

                num_images += 1

            print(f"✅ {class_name} balanced to {target_size} images.")

# Run the balancing process
print("🔍 Checking class distribution...")
before_counts = check_class_distribution(dataset_path)
print("Before Balancing:", before_counts)

balance_dataset(dataset_path)

after_counts = check_class_distribution(dataset_path)
print("✅ After Balancing:", after_counts)

🔍 Checking class distribution...
Before Balancing: {'Calculus': 613, 'Caries': 618, 'Gingivitis': 619, 'healthy': 611}
✅ After Balancing: {'Calculus': 613, 'Caries': 618, 'Gingivitis': 619, 'healthy': 611}


In [65]:
pip install tensorflow numpy matplotlib opencv-python

Note: you may need to restart the kernel to use updated packages.


In [66]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
import matplotlib.pyplot as plt
import numpy as np
import os

In [67]:
# Define dataset path
dataset_path = "C:\\Users\\megha\\OneDrive\\Desktop\\Major Project\\Dataset\\multi_dataset"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Define Image Augmentation & Normalization
datagen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2  # 80% Training, 20% Validation
)

# Load Training Data
train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

# Load Validation Data
val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

# Get Class Labels
print("Class Indices:", train_data.class_indices)

Found 1971 images belonging to 4 classes.
Found 490 images belonging to 4 classes.
Class Indices: {'Calculus': 0, 'Caries': 1, 'Gingivitis': 2, 'healthy': 3}


In [69]:
# Load Pretrained MobileNetV2 Model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
base_model.trainable = False  # Freeze Pretrained Layers

# Build MobileNetV2 Model
mobilenet_model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(4, activation="softmax")  # 4 classes: Healthy, Caries, Calculus, Hypodontia
])

# Compile Model
mobilenet_model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Train MobileNetV2 Model
EPOCHS = 50
mobilenet_history = mobilenet_model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS
)

# Save Model
mobilenet_model.save("mobilenet_dental_model.h5")

Epoch 1/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 75s 1s/step - accuracy: 0.5102 - loss: 1.1981 - val_accuracy: 0.7408 - val_loss: 0.6440
Epoch 2/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 42s 671ms/step - accuracy: 0.7590 - loss: 0.5825 - val_accuracy: 0.6959 - val_loss: 0.6667
Epoch 3/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 47s 753ms/step - accuracy: 0.8109 - loss: 0.4953 - val_accuracy: 0.7449 - val_loss: 0.5840
Epoch 4/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 41s 665ms/step - accuracy: 0.8245 - loss: 0.4067 - val_accuracy: 0.8122 - val_loss: 0.4516
Epoch 5/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 41s 663ms/step - accuracy: 0.8567 - loss: 0.3515 - val_accuracy: 0.8102 - val_loss: 0.4392
Epoch 6/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 34s 552ms/step - accuracy: 0.8863 - loss: 0.2959 - val_accuracy: 0.8000 - val_loss: 0.4799
Epoch 7/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 33s 532ms/step - accuracy: 0.8974 - loss: 0.2861 - val_accuracy: 0.8306 - val_loss: 0.4061
Epoch 8/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 33s 537ms/step - accuracy: 0.8939 - loss: 0.2640 - val_accurac

In [70]:
# Evaluate MobileNetV2
mobilenet_loss, mobilenet_accuracy = mobilenet_model.evaluate(val_data)
print(f"MobileNetV2 - Validation Accuracy: {mobilenet_accuracy:.4f}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 441ms/step - accuracy: 0.8192 - loss: 0.4681
MobileNetV2 - Validation Accuracy: 0.8184


In [126]:
import cv2
from tensorflow.keras.preprocessing import image

# Load Model
model = tf.keras.models.load_model("mobilenet_dental_model.h5")

# Define Class Labels
class_labels = ["Calculus", "Caries", "Gingivitis","Healthy"]

# Function to Predict on New Images
def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    prediction = model.predict(img_array)
    predicted_class = class_labels[np.argmax(prediction)]
    confidence_score = np.max(prediction)

    print(f"Predicted Class: {predicted_class} (Confidence: {confidence_score:.4f})")

# Predict on a Sample Image
img_path = "C:\\Users\\megha\\OneDrive\\Desktop\\Major Project\\Dataset\\test\\ca1.jpg"
predict_image(img_path, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted Class: Calculus (Confidence: 0.9987)
